# Transmission WAXS Analysis for Operando Membrane Fouling Experiments
For single (i.e., non-tiled) images collected at ALS beamline 7.3.3

In [ ]:
import warnings; warnings.filterwarnings('ignore')

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline 
import pyFAI
import fabio
plt.style.use('default')
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.colorbar import Colorbar as colorbar
from matplotlib.colors import LogNorm
from scipy.integrate import simps
from tqdm import tqdm

In [ ]:
flag_save = True

In [ ]:
# using  sorted, sample10 comes before sample1, so we clean this up now - taken from: https://stackoverflow.com/questions/5967500/how-to-correctly-sort-a-string-with-a-number-inside
def text_to_int(text): return int(text) if text.isdigit() else text
def natural_keys(text): return [ text_to_int(c) for c in re.split(r'(\d+)', text ) ]

In [ ]:
# locate directory

master_path = '/global/cfs/cdirs/als/mwet/data/Landsman Temp/operando instrument paper/'
sample_folder = 'exp01'  

data_path = os.path.join(master_path, 'waxs_data', sample_folder)
save_path = os.path.join(master_path, 'waxs_analysis', sample_folder)
if flag_save: os.makedirs(save_path, exist_ok=True)
    

In [ ]:
# load data, including any of the "restarts" when the hutch needed to be opened and data acquisition was paused

files = sorted([os.path.join(data_path, f) for f in os.listdir(os.path.join(data_path)) if 'restart' not in f and 'beamstop_test' not in f and 'autoexpose_test' not in f and 'DS_Store' not in f])
files_restart = sorted([os.path.join(data_path, f) for f in os.listdir(os.path.join(data_path)) if 'restart_' in f and 'beamstop_test' not in f and 'autoexpose_test' not in f and 'DS_Store' not in f])
files_restart2 = sorted([os.path.join(data_path, f) for f in os.listdir(os.path.join(data_path)) if 'restart2' in f and 'beamstop_test' not in f and 'autoexpose_test' not in f and 'DS_Store' not in f])
files.sort(key = natural_keys)
files_restart.sort(key = natural_keys)
files_restart2.sort(key = natural_keys)
files += files_restart
files += files_restart2

imagefiles = sorted([os.path.join(data_path, f) for f in files if '.edf' in f], key=lambda x:  files.index(x))
txtfiles = sorted([os.path.join(data_path, f) for f in files if '.txt' in f], key=lambda x:  files.index(x))
sample_list = pd.Series([f[ : f.find('_2m.edf')] for f in imagefiles]).unique()
sample_list_fname = pd.Series([f[f.find(data_path)+len(data_path)+1:] for f in sample_list]).unique()

print(pd.Series(sample_list_fname))

In [ ]:
# plot 2D images

for samp in tqdm((sample_list[:])):
    fname = sorted(f for f in imagefiles if (samp in f))[0]    
    file = os.path.join(data_path,fname)
    image = fabio.open(file)
    array = image.data
    array[array < 1] = 1

    fig, ax = plt.subplots()
    im = ax.imshow(array, norm=LogNorm(vmin=np.percentile(array, 20), vmax=np.percentile(array, 99.9)))

    sample_title = os.path.basename(fname)
    ax.set_title(sample_title)
    ax.axis('off')
        
    ax_divider = make_axes_locatable(ax)
    cax = ax_divider.append_axes('bottom', size='3%', pad='2%')
    plt.colorbar(im, cax=cax, orientation='horizontal')
    cax.set_xlabel('Scattering intensity (arbitrary units)', size=8)
    cax.xaxis.set_label_position('bottom')
    cax.xaxis.tick_bottom()
    cax.xaxis.set_tick_params(labelsize=8)   
    fig.tight_layout()
    
    if flag_save:
        save_path_2dimages = os.path.join(save_path, 'waxs_2dimages')
        os.makedirs(save_path_2dimages, exist_ok=True)
        save_fname = os.path.join(save_path_2dimages, os.path.basename(samp) + '_2dimage.png')
        fig.savefig(save_fname, bbox_inches='tight', dpi=300)    
    

In [ ]:
# azimuthal integrator (this is the most computationally expensive component, so we load it once and refer to it within loops later)
# poni file was created using pyFAI calibration GUI (https://pyfai.readthedocs.io/en/master/usage/cookbook/calib-gui/index.html)

poni = os.path.join(master_path, 'waxs_data', 'waxs_poni_mrl.poni')
ai = pyFAI.load(poni)
ai.maskfile = os.path.join(master_path, 'waxs_data', 'waxs_mask_mrl.edf')

ai

In [ ]:
# batch process(+save) reduced profiles --> normalize based on diode value, perform bkg subtraction, radially integrate, plot 1D profiles    

colormap = plt.get_cmap('jet')
colors = colormap(np.linspace(0.1, 1, len(sample_list)))
azimuth_range=[-179,179]

fig, ax = plt.subplots(figsize=(6, 5))

transmission_list = []

for idx, samp in tqdm(enumerate(sample_list[:])):
    
    # define ctrl/bkg scans
    ctrl_fname = sorted(f for f in os.listdir(os.path.join(data_path)) if f'{sample_folder}_10s_1_2m' in f and '.edf' in f)[0]
    ctrl_txtfile = sorted(f for f in os.listdir(os.path.join(data_path)) if f'{sample_folder}_10s_1_2m' in f and '.txt' in f)[0]

    npt=4000
    
    ctrl_I1_value = int(np.loadtxt(os.path.join(data_path, ctrl_txtfile), skiprows=1, max_rows=1))
    ctrl_diode_value = int(np.loadtxt(os.path.join(data_path, ctrl_txtfile), skiprows=0, max_rows=1))
    ctrl_file = os.path.join(data_path, ctrl_fname)
    ctrl_image = fabio.open(ctrl_file)
    ctrl_array = ctrl_image.data
    ctrl_array[ctrl_array < 10] = 1
    ctrl_reduced = ai.integrate1d(ctrl_array, npt=npt, azimuth_range=azimuth_range)    
    q_ctrl = ctrl_reduced[0] / 10
    i_ctrl = ctrl_reduced[1] / ctrl_I1_value

    # load txt file for normalization values
    txtfile = sorted(t for t in txtfiles if (samp in t))[0]
    I1_value = int(np.loadtxt(txtfile, skiprows=1, max_rows=1))
    diode_value = int(np.loadtxt(txtfile, skiprows=0, max_rows=1))

    
    # load and reduce scattering profiles
    fname = sorted(f for f in files if (samp in f))[0]
    sample_file = os.path.join(data_path,os.path.basename(fname))
    sample_image = fabio.open(sample_file)
    sample_array = sample_image.data
    sample_array[sample_array < 1] = 1
    sample_reduced = ai.integrate1d(sample_array, npt=npt, azimuth_range=azimuth_range)    
    q_sample = sample_reduced[0] / 10
    i_sample = sample_reduced[1] / I1_value   
   # i_sample = sample_reduced[1] / diode_value
    i_sample_norm = i_sample #- i_ctrl
    
 

    # plot time series
    ax.plot(q_sample,i_sample_norm,'-', linewidth=1, markerfacecolor='none', label=samp, color=colors[idx])
    ax.set_title(f'Radially integrated WAXS profile\n{sample_folder}')
    ax.set_xlabel(r'Scattering vector, $q \ (\mathrm{\AA^{-1}})$')
    ax.set_ylabel('Scattering intensity (arbitrary units)')
    ax.grid(which='major', color='lightgrey', linewidth=0.6); ax.grid(which='minor', color='lightgrey', linewidth=0.3)
    #ax.set_xscale('log')
    #ax.set_xlim([2.04, 2.12])
    #ax.set_xlim([1.6,1.7])
    #ax.set_xlim([2.5,2.55])
    #ax.set_xlim([2.9, 3.1])
    #ax.set_xlim([3.3,3.36])
    #ax.set_xlim([1.5,3.1])
    #ax.legend() #bbox_to_anchor=(1.04, 1))
    
    
    # lets track the transmission flux throughout the data series
    transmission_flux = diode_value / I1_value
    transmission_list.append(transmission_flux)

    
    
    if flag_save:        
        save_path_radint = os.path.join(save_path, 'waxs_reduced')
        os.makedirs(save_path_radint, exist_ok=True)
        save_plotname = os.path.join(save_path, sample_folder + '_radint_series.png')
        fig.savefig(save_plotname, bbox_inches='tight', dpi=300)
        save_txtname = os.path.join(save_path_radint, sample_folder + '_' + str(idx) + '_radint.txt')
        np.savetxt(save_txtname, np.vstack((q_sample, i_sample_norm)).T) 
        save_transmission = os.path.join(save_path, sample_folder + '_transmissionflux.txt')
        np.savetxt(save_transmission, np.vstack((transmission_list)).T)
           

In [ ]:
# lets take a look at the transmission signal on the diode as a function of time

fig, ax = plt.subplots(figsize=(6, 4))
ax.set_title(f'Transmission flux through flow cell\n{sample_folder}')
ax.plot(transmission_list, 'o-')
ax.set_xlabel('sample index')
ax.set_ylabel('photodiode flux / I1 flux')
ax.grid(which='major', color='lightgrey', linewidth=0.6); ax.grid(which='minor', color='lightgrey', linewidth=0.3)

if flag_save:
    save_plotname = os.path.join(save_path, sample_folder + '_transmissionflux.png')
    fig.savefig(save_plotname, bbox_inches='tight', dpi=300)
        


In [ ]:
# now load the reduced files for further analysis

if not flag_save:
    save_path_radint = os.path.join(save_path, 'waxs_reduced')

reduced_path = save_path_radint

reduced_files = sorted(f for f in os.listdir(reduced_path) if '.txt' in f)
reduced_files.sort(key=natural_keys)
reduced_files


In [ ]:
# lets track the total scattering intensity

tsi_list = []

fig, ax = plt.subplots(figsize=(6, 4))

for file in tqdm(reduced_files[:]):
    
    file_path = os.path.join(reduced_path, file)
    data = np.loadtxt(file_path)
    qvector = data[:,0]
    intensity = data[:,1]   
    
    # total scattering intensity
    tsi_valid_index1 = 100
    tsi_valid_index2 = 3900
    tsi_i_sample_norm = intensity[tsi_valid_index1:tsi_valid_index2]
    tsi_q_sample = qvector[tsi_valid_index1:tsi_valid_index2]
    tsi = np.trapz(tsi_i_sample_norm, tsi_q_sample)
    tsi_list.append(tsi) 
    
    # figure time :) 
    ax.plot(tsi_list, 'o-', label=f'tsi '+'$\mathrm{\AA^{-1}}$', color='tab:blue')
    ax.set_title(f'Total scattering intensity\n{sample_folder}')
    ax.set_xlabel('sample index')
    ax.set_ylabel('Total scattering intensity')
    ax.grid(which='major', color='lightgrey', linewidth=0.6); ax.grid(which='minor', color='lightgrey', linewidth=0.3)

    if flag_save:
        save_tsiplot = os.path.join(save_path, sample_folder + '_tsi.png')
        fig.savefig(save_tsiplot, bbox_inches='tight', dpi=300)       
        save_tsitxt = os.path.join(save_path, sample_folder + '_tsi.txt')
        np.savetxt(save_tsitxt, np.vstack((tsi_list)))   


In [ ]:
# now we locate a peak of interest and track its area and q-position

peak_q = 2.08 
#peak_q = 1.63
#peak_q = 2.53
#peak_q = 2.75


peak_q_list = []
peak_area_list = []

fig, ax = plt.subplots(figsize=(6, 4))

for file in tqdm(reduced_files[:]):
    
    file_path = os.path.join(reduced_path, file)
    data = np.loadtxt(file_path)
    qvector = data[:,0]
    intensity = data[:,1]   
    
    # peak identification and area calculation
    peak_idx = np.argmin(np.abs(qvector - peak_q))
    peak_window = 50
    peak_min_idx = int(peak_idx - peak_window)
    peak_max_idx = int(peak_idx + peak_window)
    peak_q_selected = qvector[peak_min_idx : peak_max_idx]
    peak_i_selected = intensity[peak_min_idx : peak_max_idx]
    peak_background = np.interp(peak_q_selected, [peak_q_selected[0], peak_q_selected[-1]], [peak_i_selected[0], peak_i_selected[-1]])
    peak_i_subtracted = peak_i_selected - peak_background
    peak_i_max = np.argmax(peak_i_subtracted)
    peak_q_max = peak_q_selected[peak_i_max]
    peak_q_list.append(peak_q_max)
    peak_area = np.trapz(peak_i_subtracted, peak_q_selected)
    peak_area_list.append(peak_area)

    # figure time :) 
    ax.plot(peak_area_list, 'o-', label=f'peak area, q={peak_q} '+'$\mathrm{\AA^{-1}}$', color='tab:red')  
    ax.set_title(f'Peak area at q={peak_q}' + '$ \mathrm{\AA^{-1}}$' + f'\n{sample_folder}')
    ax.set_xlabel('sample index')
    ax.set_ylabel(f'Peak area (arbitrary units)')
    ax.grid(which='major', color='lightgrey', linewidth=0.6); ax.grid(which='minor', color='lightgrey', linewidth=0.3)

    if flag_save:
        save_peakplot = os.path.join(save_path, sample_folder + f'_peak_q={peak_q}.png')
        fig.savefig(save_peakplot, bbox_inches='tight', dpi=300)       
        save_peaktxt = os.path.join(save_path, sample_folder + f'_peak_q={peak_q}.txt')
        np.savetxt(save_peaktxt, np.vstack((peak_area_list)))


In [ ]:
# sometimes it is helpful to see the peak that is being fitted, so i'm leaving this here
fig, ax = plt.subplots(figsize=(5, 3))

ax.plot(peak_q_selected, peak_i_selected)    
ax.plot(peak_q_selected, peak_background)
ax.set_xlabel(r'Scattering vector, $q \ (\mathrm{\AA^{-1}})$')
ax.set_ylabel('Scattering intensity\n(arbitrary units)')
ax.grid(which='major', color='lightgrey', linewidth=0.6); ax.grid(which='minor', color='lightgrey', linewidth=0.3)
#ax.plot(peak_q_selected, peak_i_subtracted)

In [ ]:
flag_save = False

In [ ]:
# now lets make a heatmap

fig, ax = plt.subplots(figsize=(12, 4))

i_sample_list = []

for idx, reduced_file in tqdm(enumerate(reduced_files[:])):
    file_path = os.path.join(save_path_radint, reduced_file)
    data = np.loadtxt(file_path)
    q_sample = data[:, 0]
    i_sample = data[:, 1]
    i_sample_list.append(i_sample)

# convert into 2D array
heatmap_data = np.vstack((i_sample_list)).T

# define min/max color values for heatmap and plot
vmin = 0.07  
vmax = 5 
heatmap = ax.imshow(heatmap_data, cmap='magma', aspect='auto', extent=[0, len(reduced_files) * 0.5, q_sample.max(), q_sample.min()], vmin=vmin, vmax=vmax)
#heatmap = ax.imshow(heatmap_data, cmap='magma', aspect='auto', extent=[0, len(reduced_files) * 0.5, q_sample.max(), q_sample.min()], vmin=vmin, vmax=vmax)
ax.set_xlabel('Time (minutes)')
ax.set_ylabel(r'Scattering vector, $q \ (\mathrm{\AA^{-1}})$')
#ax.set_title(f'Heatmap of WAXS intensities for {sample_folder}')
cbar = plt.colorbar(heatmap, extend='both', pad=0.015)  
cbar.set_label('Intensity')
plt.show()

if flag_save:
    save_heatmap = os.path.join(save_path, sample_folder + f'_heatmap.png')
    fig.savefig(save_heatmap, bbox_inches='tight', dpi=300)       

